In [3]:
import pandas as pd
import numpy as np
import os
import boto3

from config import ACCESS_KEY, SECRET_KEY, TOKEN

In [4]:
data = pd.read_csv('C:/Users/Boris/Downloads/metrics_warranty/1.csv', sep=";") #выгрузка с ID в первом столбце

list_of_columns=[
    'Дата начала',
    'Дата отправки',
    'Дата завершения',
    'Заявка принята на склад',
    'Дата начала гарантийного периода',
    'Новая заявка',
    'Начало ремонта',
    'Отложено',
    'Выходной контроль',
    'Готово к отправке'
]

for i in list_of_columns:
    data[i]=pd.to_datetime(data[i],dayfirst=True)
    
data_products = pd.read_csv('C:/Users/Boris/Downloads/metrics_warranty/2.csv', sep=";") #выгрузка с товаром в первом столбце
data_products = data_products[['Товар', 'ID']]
data_products = data_products.fillna('')

data_products_grouped = data_products.groupby("ID")["Товар"].apply(list).reset_index()
data_merged = data.merge(
    data_products_grouped,
    on='ID',
    how='left'
)

list_of_works = ['Диагностика, выходной контроль',
    'Дополнительные работы 20 мин.', 
    'Компонентный ремонт',
    'Микропроцессорный ремонт',
    'Замена корпуса Один (пластик)',
    'Замена корпуса Один С (металл)',
    'Замена задней крышки',
    'Замена объектива модуля камеры',
    'Замена модуля камеры в сборе',
    'Прошивка камеры',
    'Прошивка домофона',
    'Присвоение ID домофона',
    'Замена микрофона',
    'Замена RFID-антенны (NFC, Em-marine)',
    'Замена мембраны',
    'Замена динамика',
    'Замена стекла',
    'Замена материнской платы',
    'Замена транзистора',
    'Замена диода',
    'Замена свитча Один',
    'Замена свитча Ио',
    'Замена визнета',
    'Замена кодека',
    'Замена процессора',
    'Замена модуля BLE',
    'Восстановление токопроводящих дорожек',
    'Замена Ик-диода',
    'Замена датчика освещенности.',
    'Восстановление LAN порта'
]

for columns in list_of_works:
    data_merged[columns] = 0

cell_number = 0
for cell in data_merged['Товар']:
    for works in cell:
        if works.strip() in list_of_works:
            data_merged.loc[cell_number, works.strip()] = 1
    cell_number += 1

for colluns in list_of_works:
    data_merged[colluns] = data_merged[colluns].astype('int64')

data_merged['ID'] = data_merged['ID'].astype('str')

data_smart_processes = pd.read_csv('C:/Users/Boris/Downloads/metrics_warranty/3.csv', sep=";") #выгрузка смарт процессоы

data_smart_processes = data_smart_processes[['ID сделки','Выявленная неисправность']]
data_smart_processes['ID сделки'] = data_smart_processes['ID сделки'].astype('str')
data_smart_processes['ID сделки'] = data_smart_processes['ID сделки'].apply(lambda x: x[0:-2])
data_smart_processes_grouped = data_smart_processes.groupby('ID сделки')['Выявленная неисправность'].apply(list).reset_index()
data_smart_processes_grouped.rename(columns={'ID сделки':'ID'}, inplace=True)

C:\Users\Boris\AppData\Local\Temp\ipykernel_32\4203132495.py:19: DtypeWarning: Columns (23,25,28,39,40,41,42,55) have mixed types. Specify dtype option on import or set low_memory=False.
  data_products = pd.read_csv('C:/Users/Boris/Downloads/metrics_warranty/2.csv', sep=";") #выгрузка с товаром в первом столбце


In [5]:
data_smart_processes_grouped

,ID,Выявленная неисправность
0,17455,"[НД → СЕТЬ, ИНТЕРНЕТ: Ни камера, ни домофон не..."
1,17497,"[НД → СЕТЬ, ИНТЕРНЕТ: Ни камера, ни домофон не..."
2,17677,"[НД → ВЫЗОВЫ, КОММУТАТОР: Нет реакции на сняти..."
3,21371,"[НД → СЕТЬ, ИНТЕРНЕТ: Ни камера, ни домофон не..."
4,21373,"[НД → СЕТЬ, ИНТЕРНЕТ: Ни камера, ни домофон не..."
...,...,...
1296,50569,"[НД → ЗВУК: В домофоне нет звука голоса, есть ..."
1297,50573,"[НД → ЗВУК: В домофоне нет звука голоса, есть ..."
1298,7789,[НД → ЗАМОК (ЭМЗ): Замок не отпускает или отпу...
1299,7797,[НДО → БЕСКОНТАКТНАЯ КНОПКА ВЫХОДА: Не обеспеч...


In [6]:
list_of_defects= ['nan',
'НД → ЗВУК: В домофоне нет звука голоса, есть только системные звуки',
'НД → ЗВУК: Домофон не издает никаких звуков',
'НД → ЗВУК: Голос или звук искажен (фон, треск, свист, шум)',
'НД → ЗВУК: Голос или звук тихий даже после настройки',
'НД → ЗВУК: Нет голоса в абонентской трубке или в приложении',
'НД → ЗАМОК (ЭМЗ): Замок не отпускает или отпускает частично',
'НД → ЗАМОК (ЭМЗ): Замок не держит',
'НД → ЗАМОК (ЭМЗ): Ключ периодически или постоянно не срабатывает',
'НД → СЕТЬ, ИНТЕРНЕТ: В сеть выходит или домофон, или камера, но не вместе',
'НД → СЕТЬ, ИНТЕРНЕТ: Ни камера, ни домофон не выходит в сеть (мигает крестик)',
'НД → СЕТЬ, ИНТЕРНЕТ: Зависает при подключении интернета',
'НД → ВЫЗОВЫ, КОММУТАТОР: Нет вызова ни в какую квартиру',
'НД → ВЫЗОВЫ, КОММУТАТОР: Нет реакции на снятие абонентской трубки',
'НД → ВЫЗОВЫ, КОММУТАТОР: Не срабатывает нажатие кнопки или кнопок (цифры, вызов, сброс)',
'НД → НЕРАБОТОСПОСОБНА: Не включается после подачи питания',
'НД → НЕРАБОТОСПОСОБНА: Постоянная перезагрузка при исправном блоке питания',
'НД → НЕРАБОТОСПОСОБНА: Периодически зависает или перезагружается',
'НД → НЕРАБОТОСПОСОБНА: Коррозия платы',
'НД → НЕРАБОТОСПОСОБНА: Воздействие перенапряжения',
'НД → НЕРАБОТОСПОСОБНА: Вандализм',
'НД → КАМЕРА: Нет видеопотока с камеры (в веб-сервисах и в приложении)',
'НД → КАМЕРА: Плохое качество видео (изменение цвета, размытие, другое)',
'НД → КАМЕРА: Камера не работает, в сеть не выходит',
'НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Нет видеопотока с камеры (в веб-сервисах и в приложении)',
'НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Плохое качество видео (изменение цвета, размытие, другое)',
'НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Камера не выходить в сеть',
'НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Камера не работает, в сеть не выходит',
'НДО → БЛОК ПИТАНИЯ: Не включается',
'НДО → БЛОК ПИТАНИЯ: Не обеспечивает нужные параметры',
'НДО → БЕСКОНТАКТНАЯ КНОПКА ВЫХОДА: Не включается',
'НДО → БЕСКОНТАКТНАЯ КНОПКА ВЫХОДА: Не обеспечивает нужные параметры',
'ДРУГОЕ: Заявка на замену корпуса',
'ДРУГОЕ: Неисправности не выявлены']

data_smart_processes_grouped.loc[data_smart_processes_grouped[data_smart_processes_grouped['ID'] == 'n'].index[0],['ID']] = "0"
data_merged['ID'] = data_merged['ID'].astype(int)
data_smart_processes_grouped['ID'] = data_smart_processes_grouped['ID'].astype(int)

In [7]:
for columns in list_of_defects:
    data_smart_processes_grouped[columns] = ''

In [8]:
line_count = 0
for lines in data_smart_processes_grouped['Выявленная неисправность']:
    for defect in lines:
        data_smart_processes_grouped.loc[line_count,[defect]] = '1'
    line_count += 1

In [9]:
data_smart_processes_grouped = data_smart_processes_grouped.drop(['Выявленная неисправность','nan'], axis=1)

In [10]:
data_smart_processes_grouped

,ID,"НД → ЗВУК: В домофоне нет звука голоса, есть только системные звуки",НД → ЗВУК: Домофон не издает никаких звуков,"НД → ЗВУК: Голос или звук искажен (фон, треск, свист, шум)",НД → ЗВУК: Голос или звук тихий даже после настройки,НД → ЗВУК: Нет голоса в абонентской трубке или в приложении,НД → ЗАМОК (ЭМЗ): Замок не отпускает или отпускает частично,НД → ЗАМОК (ЭМЗ): Замок не держит,НД → ЗАМОК (ЭМЗ): Ключ периодически или постоянно не срабатывает,"НД → СЕТЬ, ИНТЕРНЕТ: В сеть выходит или домофон, или камера, но не вместе",...,"НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Плохое качество видео (изменение цвета, размытие, другое)",НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Камера не выходить в сеть,"НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Камера не работает, в сеть не выходит",НДО → БЛОК ПИТАНИЯ: Не включается,НДО → БЛОК ПИТАНИЯ: Не обеспечивает нужные параметры,НДО → БЕСКОНТАКТНАЯ КНОПКА ВЫХОДА: Не включается,НДО → БЕСКОНТАКТНАЯ КНОПКА ВЫХОДА: Не обеспечивает нужные параметры,ДРУГОЕ: Заявка на замену корпуса,ДРУГОЕ: Неисправности не выявлены,NaN
0,17455,,,,,,,,,,...,,,,,,,,,,NaN
1,17497,,,,,,,,,,...,,,,,,,,,,NaN
2,17677,,,,,,,,,,...,,,,,,,,,,NaN
3,21371,,,,,,,,,,...,,,,,,,,,,NaN
4,21373,,,,,,,,,,...,,,,,,,,,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296,50569,1,,,,,,,,,...,,,,,,,,,,NaN
1297,50573,1,,,,,,,,,...,,,,,,,,,,NaN
1298,7789,,,1,,,1,,,,...,,,,,,1,,,,NaN
1299,7797,1,,,,,,,,,...,,,,,,,1,,,NaN


In [11]:
data_smart_processes_grouped.to_excel('C:/Users/Boris/Downloads/metrics_warranty/remade/data_smart_processes_grouped.xlsx',index=False)

In [12]:
data_merged = data_merged.merge(
    data_smart_processes_grouped,
    on='ID',
    how='left'
)

data_merged.fillna('')

,ID,Стадия сделки,Статус оплаты,Статус доставки,Название сделки,Сумма,Дата начала,Комментарии,Серийный номер оборудования,Партнёр,...,"НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Плохое качество видео (изменение цвета, размытие, другое)",НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Камера не выходить в сеть,"НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Камера не работает, в сеть не выходит",НДО → БЛОК ПИТАНИЯ: Не включается,НДО → БЛОК ПИТАНИЯ: Не обеспечивает нужные параметры,НДО → БЕСКОНТАКТНАЯ КНОПКА ВЫХОДА: Не включается,НДО → БЕСКОНТАКТНАЯ КНОПКА ВЫХОДА: Не обеспечивает нужные параметры,ДРУГОЕ: Заявка на замену корпуса,ДРУГОЕ: Неисправности не выявлены,NaN
0,50589,Новая заявка,,,Обращение по гарантии панели № 2236150079 от О...,,2024-05-16,,2236150079,077fdc46-d894-4d3c-a93e-b219bc1ab331,...,,,,,,,,,,
1,50547,Заявка готова к отправке,,,Обращение по гарантии панели № 2018111743 от Ц...,,2024-05-15,,2018111743,495b2f5e-8835-49a3-9c20-3571a8003adb,...,,,,,,,,,,
2,50545,Заявка готова к отправке,,,Обращение по гарантии панели № 1944104381 от Ц...,,2024-05-15,,1944104381,495b2f5e-8835-49a3-9c20-3571a8003adb,...,,,,,,,,,,
3,50543,Заявка готова к отправке,,,Обращение по гарантии панели № 2012105459 от Ц...,,2024-05-15,,2012105459,495b2f5e-8835-49a3-9c20-3571a8003adb,...,,,,,,,,,,
4,50541,Заявка готова к отправке,,,Обращение по гарантии панели № 2018113097 от Ц...,,2024-05-15,,2018113097,495b2f5e-8835-49a3-9c20-3571a8003adb,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10360,9353,Ошибка заведения,,,Сделка #9353,0.0,2021-10-19,,,,...,,,,,,,,,,
10361,9187,Ошибка заведения,,,0111,0.0,2021-10-02,,,,...,,,,,,,,,,
10362,8783,Ошибка заведения,,,"Заполнение CRM-формы ""Обращение по гарантии""",0.0,2021-08-25,,,,...,,,,,,,,,,
10363,7797,Ошибка заведения,,,Тест 001,1770.0,2021-04-16,,2018113581,,...,,,,,,,1,,,


In [13]:
data_merged.to_excel('C:/Users/Boris/Downloads/metrics_warranty/remade/data_merged.xlsx',index=False)
data_merged

,ID,Стадия сделки,Статус оплаты,Статус доставки,Название сделки,Сумма,Дата начала,Комментарии,Серийный номер оборудования,Партнёр,...,"НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Плохое качество видео (изменение цвета, размытие, другое)",НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Камера не выходить в сеть,"НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Камера не работает, в сеть не выходит",НДО → БЛОК ПИТАНИЯ: Не включается,НДО → БЛОК ПИТАНИЯ: Не обеспечивает нужные параметры,НДО → БЕСКОНТАКТНАЯ КНОПКА ВЫХОДА: Не включается,НДО → БЕСКОНТАКТНАЯ КНОПКА ВЫХОДА: Не обеспечивает нужные параметры,ДРУГОЕ: Заявка на замену корпуса,ДРУГОЕ: Неисправности не выявлены,NaN
0,50589,Новая заявка,NaN,NaN,Обращение по гарантии панели № 2236150079 от О...,NaN,2024-05-16,NaN,2236150079,077fdc46-d894-4d3c-a93e-b219bc1ab331,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50547,Заявка готова к отправке,NaN,NaN,Обращение по гарантии панели № 2018111743 от Ц...,NaN,2024-05-15,NaN,2018111743,495b2f5e-8835-49a3-9c20-3571a8003adb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,50545,Заявка готова к отправке,NaN,NaN,Обращение по гарантии панели № 1944104381 от Ц...,NaN,2024-05-15,NaN,1944104381,495b2f5e-8835-49a3-9c20-3571a8003adb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,50543,Заявка готова к отправке,NaN,NaN,Обращение по гарантии панели № 2012105459 от Ц...,NaN,2024-05-15,NaN,2012105459,495b2f5e-8835-49a3-9c20-3571a8003adb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,50541,Заявка готова к отправке,NaN,NaN,Обращение по гарантии панели № 2018113097 от Ц...,NaN,2024-05-15,NaN,2018113097,495b2f5e-8835-49a3-9c20-3571a8003adb,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10360,9353,Ошибка заведения,NaN,NaN,Сделка #9353,0.0,2021-10-19,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10361,9187,Ошибка заведения,NaN,NaN,0111,0.0,2021-10-02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10362,8783,Ошибка заведения,NaN,NaN,"Заполнение CRM-формы ""Обращение по гарантии""",0.0,2021-08-25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10363,7797,Ошибка заведения,NaN,NaN,Тест 001,1770.0,2021-04-16,NaN,2018113581,NaN,...,,,,,,,1,,,NaN


In [14]:
list_of_defects3 = [
    'Выявлено: Звук',
    'Выявлено: Замок (ЭМЗ)',
    'Выявлено: Сеть, Интернет',
    'Выявлено: Вызовы, коммутатор',
    'Выявлено: Неработоспособна',
    'Выявлено: Камера',
    'Выявлено: Неисправность Домофона: Другое',
    'Выявлено: Камера видеонаблюдения',
    'Выявлено: Блок питания',
    'Выявлено: Бесконтактная кнопка выхода',
    'Выявлено: Неполадки другого оборудования: Другое']

data_merged[list_of_defects3] = data_merged[list_of_defects3].fillna('')
rows_count = data_merged.shape[0]
    

for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НД → ЗВУК: В домофоне нет звука голоса, есть только системные звуки'] == '1' or\
        data_merged.loc[cell_number,'НД → ЗВУК: Домофон не издает никаких звуков'] == '1' or\
        data_merged.loc[cell_number,'НД → ЗВУК: Голос или звук искажен (фон, треск, свист, шум)'] == '1' or\
        data_merged.loc[cell_number,'НД → ЗВУК: Голос или звук тихий даже после настройки'] == '1' or\
        data_merged.loc[cell_number,'НД → ЗВУК: Нет голоса в абонентской трубке или в приложении'] == '1':
        data_merged.loc[cell_number,'Выявлено: Звук'] = '1'
       

for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НД → СЕТЬ, ИНТЕРНЕТ: В сеть выходит или домофон, или камера, но не вместе'] == '1':
        data_merged.loc[cell_number,'Выявлено: Сеть, Интернет'] = '1'
    elif data_merged.loc[cell_number,'НД → СЕТЬ, ИНТЕРНЕТ: Ни камера, ни домофон не выходит в сеть (мигает крестик)'] == '1':
        data_merged.loc[cell_number,'Выявлено: Сеть, Интернет'] = '1'
    elif data_merged.loc[cell_number,'НД → СЕТЬ, ИНТЕРНЕТ: Зависает при подключении интернета'] == '1':
        data_merged.loc[cell_number,'Выявлено: Сеть, Интернет'] = '1'

    
for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НД → ЗАМОК (ЭМЗ): Замок не отпускает или отпускает частично'] == '1' or\
    data_merged.loc[cell_number,'НД → ЗАМОК (ЭМЗ): Замок не держит'] == '1' or\
    data_merged.loc[cell_number,'НД → ЗАМОК (ЭМЗ): Ключ периодически или постоянно не срабатывает'] == '1':
        data_merged.loc[cell_number,'Выявлено: Замок (ЭМЗ)'] = '1'


for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НД → ВЫЗОВЫ, КОММУТАТОР: Нет вызова ни в какую квартиру'] == '1' or\
    data_merged.loc[cell_number,'НД → ВЫЗОВЫ, КОММУТАТОР: Нет реакции на снятие абонентской трубки'] == '1' or\
    data_merged.loc[cell_number,'НД → ВЫЗОВЫ, КОММУТАТОР: Не срабатывает нажатие кнопки или кнопок (цифры, вызов, сброс)'] == '1':
        data_merged.loc[cell_number,'Выявлено: Вызовы, коммутатор'] = '1'

       
for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НД → НЕРАБОТОСПОСОБНА: Не включается после подачи питания'] == '1' or\
    data_merged.loc[cell_number,'НД → НЕРАБОТОСПОСОБНА: Постоянная перезагрузка при исправном блоке питания'] == '1' or\
    data_merged.loc[cell_number,'НД → НЕРАБОТОСПОСОБНА: Периодически зависает или перезагружается'] == '1' or\
    data_merged.loc[cell_number,'НД → НЕРАБОТОСПОСОБНА: Коррозия платы'] == '1' or\
    data_merged.loc[cell_number,'НД → НЕРАБОТОСПОСОБНА: Воздействие перенапряжения'] == '1' or\
    data_merged.loc[cell_number,'НД → НЕРАБОТОСПОСОБНА: Вандализм'] == '1':
        data_merged.loc[cell_number,'Выявлено: Неработоспособна'] = '1'
       

for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НД → КАМЕРА: Нет видеопотока с камеры (в веб-сервисах и в приложении)'] == '1' or\
    data_merged.loc[cell_number,'НД → КАМЕРА: Плохое качество видео (изменение цвета, размытие, другое)'] == '1' or\
    data_merged.loc[cell_number,'НД → КАМЕРА: Камера не работает, в сеть не выходит'] == '1':
        data_merged.loc[cell_number,'Выявлено: Камера'] = '1'


for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Нет видеопотока с камеры (в веб-сервисах и в приложении)'] == '1' or\
    data_merged.loc[cell_number,'НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Плохое качество видео (изменение цвета, размытие, другое)'] == '1' or\
    data_merged.loc[cell_number,'НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Камера не выходить в сеть'] == '1' or\
    data_merged.loc[cell_number,'НДО → КАМЕРА ВИДЕОНАБЛЮДЕНИЯ: Камера не работает, в сеть не выходит'] == '1':
        data_merged.loc[cell_number,'Выявлено: Камера видеонаблюдения'] = '1'

        
for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НДО → БЛОК ПИТАНИЯ: Не включается'] == '1' or\
    data_merged.loc[cell_number,'НДО → БЛОК ПИТАНИЯ: Не обеспечивает нужные параметры'] == '1':
        data_merged.loc[cell_number,'Выявлено: Блок питания'] = '1'

        
for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'НДО → БЕСКОНТАКТНАЯ КНОПКА ВЫХОДА: Не включается'] == '1' or\
    data_merged.loc[cell_number,'НДО → БЕСКОНТАКТНАЯ КНОПКА ВЫХОДА: Не обеспечивает нужные параметры'] == '1':
        data_merged.loc[cell_number,'Выявлено: Бесконтактная кнопка выхода'] = '1'

for cell_number in range(0,rows_count):
    if data_merged.loc[cell_number,'ДРУГОЕ: Заявка на замену корпуса'] == '1' or\
    data_merged.loc[cell_number,'ДРУГОЕ: Заявка на замену корпуса'] == '1':
        data_merged.loc[cell_number,'Выявлено: Неполадки другого оборудования: Другое'] = '1'


In [15]:
data_merged.to_excel('C:/Users/Boris/Downloads/metrics_warranty/remade/metrics_warranty3.xlsx', index=False)

In [16]:
data_merged.to_csv('C:/Users/Boris/Downloads/metrics_warranty/remade/metrics_warranty.csv', index=False, sep=',')

In [17]:
ACCESS_KEY = ACCESS_KEY
SECRET_KEY = SECRET_KEY
BUCKET_NAME = "dwh-asgard"
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow")


def get_s3_instance():
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )
    
def upload_dump_to_s3():
    get_s3_instance().upload_file(
        Filename= f'C:/Users/Boris/Downloads/metrics_warranty/remade/metrics_warranty.csv',
        Bucket=BUCKET_NAME,
        Key=f"metrics_dir_warranty/metrics_warranty.csv"
    )

get_s3_instance()
upload_dump_to_s3()